<a href="https://colab.research.google.com/github/HerlanAssis/coisas-que-aprendi/blob/main/eduardo_mendes/160_Peewee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# \[\#160\] Coisas que Aprendi com Eduardo Mendes
Coisas que aprendi com Eduardo Mendes e como poderia usar esse conhecimento no mundo real?

------

Na live de python [#160 - Peewee ORM, simplificando banco de dados](https://www.youtube.com/watch?v=fJixdxPOz-E) Eduardo Mender apresentou o Pewee ORM e como é simples trabalhar ele.

Depois que assistir o vídeo, fiquei encantado e resolvi aplicar o conhecimento adquirido em caso comum no mundo real.

### Tarefa:
> Criar e popular um banco de dados relacional a partir de um arquivo CSV.

Nesse momento você pode até pensar: "já existe ferramentas pra isso!". De fato, existe inúmeras maneiras de fazer isso mas, neste exemplo, vamos utilizar Python, leitura de arquivo e uma ferramenta incrível, o Peewee.

Temos um arquivos CSV com dados referentes a uma pesquisa direta com o cliente final e queremos estruturar tudo isso no nosso branco de dados. Nosso arquivo CSV tem a seguinte estrutura:

|contrato|nome     |data de ativação|data de débito|valor débito|data pagamento|status |foi pago|
|--------|---------|----------------|--------------|------------|--------------|-------|--------|
|1234    |Cliente 1|24/03/20        |24/01/21      |R\$ 150,00   |24/03/21      |aberto |não     |
|4321    |Cliente 2|24/03/20        |24/01/21      |R\$ 80,00    |24/03/21      |fechado|sim     |
|1598    |Cliente 3|24/03/20        |24/01/21      |R\$ 75,00    |24/03/21      |aberto |não     |

Abstraindo isso, podemos criar pelo menos duas classes, ```Pesquisa``` e ```StatusPesquisa```.

Considere as colunas "contrato", "nome", "data de ativação", "data de débito", "valor débito", "data pagamento" como características de uma classe ```Pesquisa``` e as colunas "status" e "foi pago" como características da classe ```StatusPesquisa```.

vamos criar o nosso csv com a ajuda do *colab magic commands*

In [1]:
%%writefile teste.csv
contrato,nome,data de ativação,data de débito,valor débito,data pagamento,status,foi pago
1234,Cliente 1,24/03/20,24/01/21,"R$ 150,00",24/03/21,aberto,não
4321,Cliente 2,24/03/20,24/01/21,"R$ 80,00",24/03/21,fechado,sim
1598,Cliente 3,24/03/20,24/01/21,"R$ 75,00",24/03/21,aberto,não

Writing teste.csv


agora, fazemos a leitura do nosso arquivo csv e criamos uma estrutura simples de trabalhar, uma lista de dicionários. Sendo a uma linha equivalente a uma linha do arquivo csv e o dicionário a o registro dessa linha.

In [2]:
import csv

map_cols = {
    # 'col_name': col_position
    'contrato': 0,
    'nome': 1,
    'data_ativacao': 2,
    'data_debito': 3,
    'valor_debito': 4,
    'data_pagamento': 5,
    'status': 6,
    'foi_pago': 7
}

rows = []

with open('teste.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    # first line is for headers    
    for row in list(csv_reader)[1:]:
      new_data = {}
      for key, value in map_cols.items():
        new_data[key] = row[value]
        rows.append(new_data)

Aqui a magia acontece! Instalamos o Peewee através do pip, importamos as bibliotecas necessárias, definimos nossas tabelas utilizando classes e no fim as criamos. Top né?

In [3]:
!pip install peewee

from datetime import datetime
from peewee import (SqliteDatabase, Model, CharField, BooleanField,
                    DateField, FloatField,ForeignKeyField)

db = SqliteDatabase('pesquisa.db')


class BaseModel(Model):
  class Meta:
    database = db


class StatusPesquisa(BaseModel):
    foi_pago = BooleanField()
    status = CharField(unique=True)


class Pesquisa(BaseModel):
  status = ForeignKeyField(StatusPesquisa, backref='pesquisas')
  contrato = CharField()
  cliente = CharField(unique=True)
  data_ativacao = DateField()
  data_debito = DateField()
  valor_debito = FloatField()


db.create_tables([StatusPesquisa, Pesquisa])

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


agora iteramos sobre as nossas linhas e populamos nossas tabelas.

PS: não façam isso em produção.

In [4]:
for row in rows:  
  try:
    StatusPesquisa.create(
        foi_pago=row['foi_pago']=='sim',
        status=row['status']
    )
  except:
    ...
  try:
    Pesquisa.create(
        status=StatusPesquisa.select().where(StatusPesquisa.status==row['status']),
        contrato=row['contrato'],
        cliente=row['nome'],
        data_ativacao=row['data_ativacao'],
        data_debito=row['data_debito'],
        valor_debito=row['valor_debito']
    )
  except:
    ...

e fim, temos nossos dados em nosso banco :D

In [5]:
for p in Pesquisa.select():
  print(f'Cliente: {p.cliente}, status da pesquisa: {p.status.get().status}')

Cliente: Cliente 1, status da pesquisa: aberto
Cliente: Cliente 2, status da pesquisa: aberto
Cliente: Cliente 3, status da pesquisa: aberto
